In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
ls -lh "/content/drive/MyDrive/Colab Notebooks/"

total 587M
-rw------- 1 root root 471K Apr  2 02:41 Animals.ipynb
-rw------- 1 root root 586M Mar 31 21:35 archive.zip
-rw------- 1 root root 486K Apr  1 21:54 Yolov8.ipynb


In [3]:
!pip install ultralytics opencv-python pandas
!pip install ultralytics --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.1/977.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [4]:
!unzip "/content/drive/MyDrive/Colab Notebooks/archive.zip" -d /content/archive

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: /content/archive/raw-img/ragno/OIP-MVQChNQpHiMZEVMPehrvoAHaE6.jpeg  
  inflating: /content/archive/raw-img/ragno/OIP-MVdIFDAFpNyK_XCpwkDIWgHaFj.jpeg  
  inflating: /content/archive/raw-img/ragno/OIP-MYY_8k5AHqMJwTCNXfH6rwHaFj.jpeg  
  inflating: /content/archive/raw-img/ragno/OIP-MZEVXuoMsATG8u_UuTQ6hAHaE8.jpeg  
  inflating: /content/archive/raw-img/ragno/OIP-MZg0tznDRE95AMfyuutuYAHaE6.jpeg  
  inflating: /content/archive/raw-img/ragno/OIP-M_r9DkR2X5gP0P5-z8wPYQHaE7.jpeg  
  inflating: /content/archive/raw-img/ragno/OIP-M_xDUSzjlF3-w9XHCm93cgHaFF.jpeg  
  inflating: /content/archive/raw-img/ragno/OIP-MbI4-bMyutRX3X382IAVcQAAAA.jpeg  
  inflating: /content/archive/raw-img/ragno/OIP-MbMksZdD7gomsC9fd6cpKgHaHJ.jpeg  
  inflating: /content/archive/raw-img/ragno/OIP-Md4AMcvjaHcivJqeW06wEwHaFW.jpeg  
  inflating: /content/archive/raw-img/ragno/OIP-MgCgailkT5XkO1XczzQCngHaLK.jpeg  
  inflating: /content/archive/

In [5]:
import os
import shutil
import importlib.util
from sklearn.model_selection import train_test_split

dataset_path = "/content/archive/raw-img/"
output_path = "/content/archive_yolo/"
translate_path = "/content/archive/translate.py"

spec = importlib.util.spec_from_file_location("translate", translate_path)
translate_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(translate_module)

class_map = translate_module.translate

class_map = {k: v for k, v in class_map.items() if k in os.listdir(dataset_path)}

for split in ['train', 'valid', 'test']:
    for class_name in class_map.values():
        os.makedirs(os.path.join(output_path, split, class_name), exist_ok=True)

for it_class, en_class in class_map.items():
    img_path = os.path.join(dataset_path, it_class)
    if not os.path.exists(img_path):
        continue

    images = os.listdir(img_path)
    train_files, temp_files = train_test_split(images, test_size=0.2, random_state=42)  # 80% train, 20% temp
    valid_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)  # 10% valid, 10% test

    for file in train_files:
        shutil.copy(os.path.join(img_path, file), os.path.join(output_path, "train", en_class, file))

    for file in valid_files:
        shutil.copy(os.path.join(img_path, file), os.path.join(output_path, "valid", en_class, file))

    for file in test_files:
        shutil.copy(os.path.join(img_path, file), os.path.join(output_path, "test", en_class, file))

print("Данные успешно разделены: 80% train, 10% valid, 10% test!")

✅ Данные успешно разделены: 80% train, 10% valid, 10% test!


In [6]:
import torch
print(torch.cuda.is_available())


False


In [7]:
yaml_content = """path: /content/archive_yolo
train: /content/archive_yolo/train
val: /content/archive_yolo/valid
test: /content/archive_yolo/test
'nc': 10,

names: ['dog', 'horse', 'elephant', 'butterfly', 'chicken', 'cat', 'cow', 'sheep', 'spider', 'squirrel']
"""

with open("/content/archive_yolo/data.yaml", "w") as f:
    f.write(yaml_content)

✅ Файл data.yaml создан!


In [9]:
from ultralytics import YOLO

model = YOLO("yolov8s-cls.pt")

model.train(data="/content/archive_yolo", epochs=15, imgsz=224, device="cpu")


Ultralytics 8.3.100 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/archive_yolo, epochs=15, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

train: Scanning /content/archive_yolo/train... 17082 images, 0 corrupt: 100%|██████████| 17082/17082 [00:02<00:00, 5778.79it/s]


train: New cache created: /content/archive_yolo/train.cache


val: Scanning /content/archive_yolo/test... 2140 images, 0 corrupt: 100%|██████████| 2140/2140 [00:00<00:00, 6103.29it/s]

val: New cache created: /content/archive_yolo/test.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)


...

                   all      0.974      0.997

15 epochs completed in 6.956 hours.


Optimizer stripped from runs/classify/train/weights/last.pt, 10.3MB
Optimizer stripped from runs/classify/train/weights/best.pt, 10.3MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.100 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
YOLOv8s-cls summary (fused): 30 layers, 5,086,729 parameters, 0 gradients, 12.5 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/archive_yolo/train... found 17082 images in 9 classes ✅ 
val: /content/archive_yolo/test... found 2140 images in 9 classes ✅ 
test: /content/archive_yolo/test... found 2140 images in 9 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 67/67 [01:01<00:00,  1.09it/s]


                   all      0.975      0.997
Speed: 0.0ms preprocess, 25.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ed3ab569bd0>
curves: []
curves_results: []
fitness: 0.9859813153743744
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9747663736343384, 'metrics/accuracy_top5': 0.9971962571144104, 'fitness': 0.9859813153743744}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.000794142526552673, 'inference': 25.726560558410778, 'loss': 5.554813690642417e-05, 'postprocess': 0.00012565702048976792}
task: 'classify'
top1: 0.9747663736343384
top5: 0.9971962571144104

In [10]:
metrics = model.val()
print(metrics)

Ultralytics 8.3.100 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (AMD EPYC 7B12)
YOLOv8s-cls summary (fused): 30 layers, 5,086,729 parameters, 0 gradients, 12.5 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/archive_yolo/train... found 17082 images in 9 classes ✅ 
val: /content/archive_yolo/test... found 2140 images in 9 classes ✅ 
test: /content/archive_yolo/test... found 2140 images in 9 classes ✅ 


val: Scanning /content/archive_yolo/test... 2140 images, 0 corrupt: 100%|██████████| 2140/2140 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 134/134 [01:01<00:00,  2.18it/s]


                   all      0.975      0.997
Speed: 0.0ms preprocess, 25.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ed39e2a4e50>
curves: []
curves_results: []
fitness: 0.9859813153743744
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9747663736343384, 'metrics/accuracy_top5': 0.9971962571144104, 'fitness': 0.9859813153743744}
save_dir: PosixPath('runs/classify/train2')
speed: {'preprocess': 0.0013057532558836714, 'inference': 25.49880094859609, 'loss': 0.00014621495653028714, 'postprocess': 0.00030901167936412023}
task: 'classify'
top1: 0.9747663736343384
top5: 0.9971962571144104


In [15]:
!cp -r /content/runs/classify/train/weights/ /content/drive/MyDrive/weights/
print("✅ Модель сохранена на Google Диске!")

✅ Модель сохранена на Google Диске!


In [26]:
from ultralytics import YOLO

model = YOLO("/content/runs/classify/train/weights/best.pt")

metrics = model.metrics

print(metrics)


None


## Метрики по тестовому

In [25]:
from ultralytics import YOLO
from sklearn.metrics import classification_report
import torch
import os
from pathlib import Path

model = YOLO("/content/runs/classify/train/weights/best.pt")

test_dir = "/content/archive_yolo/test/"

class_names = model.names
num_classes = len(class_names)

image_paths = []
true_labels = []

for class_id, class_name in class_names.items():
    class_folder = Path(test_dir) / class_name
    if not class_folder.exists():
        continue
    for img_file in os.listdir(class_folder):
        image_paths.append(str(class_folder / img_file))
        true_labels.append(class_id)

pred_labels = []
for img_path in image_paths:
    results = model(img_path)
    pred_labels.append(results[0].probs.top1)

report = classification_report(true_labels, pred_labels, target_names=class_names.values())
print(report)

Выходные данные были обрезаны до нескольких последних строк (5000).
Speed: 3.1ms preprocess, 37.9ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/archive_yolo/test/chicken/OIP-GMTxpS9bbxoGyvaE5Nn_wgHaFf.jpeg: 224x224 chicken 1.00, cow 0.00, sheep 0.00, dog 0.00, cat 0.00, 35.3ms
Speed: 4.1ms preprocess, 35.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/archive_yolo/test/chicken/OIP-AIUE1GnVK0nNqe0dcViO6gHaFj.jpeg: 224x224 chicken 1.00, butterfly 0.00, elephant 0.00, squirrel 0.00, dog 0.00, 36.1ms
Speed: 3.2ms preprocess, 36.1ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/archive_yolo/test/chicken/OIP-clFXTPz20w8kGtlqdy5B1AHaGB.jpeg: 224x224 chicken 1.00, dog 0.00, elephant 0.00, sheep 0.00, butterfly 0.00, 34.9ms
Speed: 3.1ms preprocess, 34.9ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/archive_yolo/test/chicken/OIP-EZjjcgt

## Тестирование

In [23]:
results = model("/content/archive_yolo/test/dog/OIP--0C2kq6I68beB3DduLoxMgHaEf.jpeg")
print(results)


image 1/1 /content/archive_yolo/test/dog/OIP--0C2kq6I68beB3DduLoxMgHaEf.jpeg: 224x224 dog 1.00, butterfly 0.00, chicken 0.00, cat 0.00, sheep 0.00, 49.8ms
Speed: 6.0ms preprocess, 49.8ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
[ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: None
names: {0: 'butterfly', 1: 'cat', 2: 'chicken', 3: 'cow', 4: 'dog', 5: 'elephant', 6: 'horse', 7: 'sheep', 8: 'squirrel'}
obb: None
orig_img: array([[[ 64, 123,  92],
        [ 64, 123,  92],
        [ 63, 123,  92],
        ...,
        [ 71, 146, 100],
        [ 72, 147, 101],
        [ 72, 147, 101]],

       [[ 65, 124,  93],
        [ 65, 124,  93],
        [ 64, 124,  93],
        ...,
        [ 68, 143,  97],
        [ 69, 144,  98],
        [ 70, 145,  99]],

       [[ 67, 127,  96],
        [ 67, 127,  96],
        [ 67, 128,  94],
        ...,
        [ 66, 141,  95],
        [ 67, 142,  96],
        [ 68, 143,  97]],

       ..

## Дообучение

In [ ]:
model = YOLO("/content/runs/classify/train/weights/best.pt")

model.train(data="/content/archive_yolo", epochs=5, imgsz=224, device="cpu")